<p><font size="5"><b>CRIMES ET DELITS EN FRANCE - SIMPLON DEV. DATA </b></font></p>


<p><font size="4"><b> 0-1. PRÉPARATION DONNÉES POUR IMPORT DANS SQL - DONNEES POLICE </b></font></p>

<p><font size="3"> *SOURCE DATASET : https://www.data.gouv.fr/fr/datasets/crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012/* </font></p>

*© 2020, Gérald Bouget  (<mailto:geraldbouget@gmail.com>)

___

In [1]:
import pandas as pd
import openpyxl
from openpyxl.utils import cell
import os, zipfile
from zipfile import ZipFile
import requests
import collections
import re
#imac:
#os.chdir('/Users/admin/SFR Cloud/Documents/Simplon_DataDev_formation/Projet_final/Datasets/crimes_delits_police_gendarmerie_annuel/')

#macbookpro:
os.chdir('/Users/geraldbouget/SFR Cloud/Documents/Simplon_DataDev_formation/Projet_final/Datasets/crimes_delits_police_gendarmerie_annuel/')


## <div class="alert alert-danger"> WARNING : ICI DATASET ORIGINAL - POUR GENDARMERIE DATASET CORRIGE - CF NOTEBOOK GENDARMERIE  </div> 

In [2]:
#chargement workbook excel
wb=openpyxl.load_workbook('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012_ORIGINAL.xlsx')

In [3]:
# CHARGEMENT FICHIER IDENTIQUE QUE PRÉCEDENT (UTILISATION CODE DIFFERENTE)
wbpd=pd.ExcelFile('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012_ORIGINAL.xlsx')

## <div class="alert alert-info"> 1. Création dictionnaire Police </div> 

In [4]:
# creation dictionnaire pour inclure listes unitaires des directions de police et services  et departements pour chaque 
# année depuis 2012
def policeDic():
    police = {}
    for year in wb.sheetnames:
        if 'PN' in year:
            police[year]={'departement':[], 'direction':[], 'service':[]}
    
    # récupération des nom de chaque departement, direction, service de chaque annee à titre unitaire
    for year in police:
        for departements in wb[year]['C1':str(cell.get_column_letter(wb[year].max_column)+'1')]:
            for dep in departements:
                if dep.value not in police[year]['departement'] and dep.value!=None:
                    police[year]['departement'].append(dep.value)
    
        for directions in wb[year]['C2':str(cell.get_column_letter(wb[year].max_column)+'2')]:
            for dir in directions:
                if dir.value not in police[year]['direction'] and dir.value!=None:
                    police[year]['direction'].append(dir.value)
        
        for services in wb[year]['C3':str(cell.get_column_letter(wb[year].max_column)+'3')]:
            for serv in services:
                if serv.value not in police[year]['service'] and serv.value!=None:
                    police[year]['service'].append(serv.value)
    
    return police

In [5]:
# liste des keys principales dans Police
liste=list(policeDic())
police=policeDic()

## <div class="alert alert-info"> 2. comparaison /checks entre années noms variables multi index (dep - directions - services </div> 

In [47]:
def depDiff():
    #comparaison departement entre années
    for i in range(len(liste)-1):
        if set(police[liste[i]]['departement']) == set(police[liste[i+1]]['departement']):
            print(str(liste[i])+ ' ' + 'a les mêmes départements que' + ' ' +str(liste[i+1]))
        else:
            print(str(liste[i]) + ' n\'a pas les mêmes départements que ' +str(liste[i+1]))
    

In [48]:
depDiff()

Services PN 2012 a les mêmes départements que Services PN 2013
Services PN 2013 a les mêmes départements que Services PN 2014
Services PN 2014 a les mêmes départements que Services PN 2015
Services PN 2015 a les mêmes départements que Services PN 2016
Services PN 2016 a les mêmes départements que Services PN 2017
Services PN 2017 a les mêmes départements que Services PN 2018
Services PN 2018 a les mêmes départements que Services PN 2019


In [49]:
def dirDiff():
    for i in range(len(liste)-1):
        if set(police[liste[i]]['direction']) == set(police[liste[i+1]]['direction']):
            print(str(liste[i])+  ' a les mêmes directions que ' +str(liste[i+1]))
        else:
            print(str(liste[i])+  ' n\'a pas les mêmes directions que ' + str(liste[i+1]))


In [50]:
dirDiff()

Services PN 2012 a les mêmes directions que Services PN 2013
Services PN 2013 a les mêmes directions que Services PN 2014
Services PN 2014 n'a pas les mêmes directions que Services PN 2015
Services PN 2015 n'a pas les mêmes directions que Services PN 2016
Services PN 2016 a les mêmes directions que Services PN 2017
Services PN 2017 a les mêmes directions que Services PN 2018
Services PN 2018 n'a pas les mêmes directions que Services PN 2019


In [51]:
def servicesDiff():
    for i in range(len(liste)-1):
        if set(police[liste[i]]['service']) == set(police[liste[i+1]]['service']):
            print(str(liste[i])+ ' a les mêmes services que ' + str(liste[i+1]))
        else:
            print(str(liste[i])+ ' n\'a pas les mêmes services que ' +str(liste[i+1]))

In [52]:
servicesDiff()

Services PN 2012 n'a pas les mêmes services que Services PN 2013
Services PN 2013 n'a pas les mêmes services que Services PN 2014
Services PN 2014 n'a pas les mêmes services que Services PN 2015
Services PN 2015 n'a pas les mêmes services que Services PN 2016
Services PN 2016 n'a pas les mêmes services que Services PN 2017
Services PN 2017 n'a pas les mêmes services que Services PN 2018
Services PN 2018 n'a pas les mêmes services que Services PN 2019


In [53]:
def serviceDoublons():
    df_doublon=pd.DataFrame({
    'nomDoubon':[],
    'numeroColonne':[],
    'annee':[]
    })
    # obligé de créer liste vide pour récuprer doublons car je parse direct colonnes je ne les verra
    # car df.columns va modifier nom variable
    listeservice=[] 
    numerocol=[]
    #nomdoublon=[]
    for name in police:
        for services in wb[name]['C3':str(openpyxl.utils.cell.get_column_letter(wb[name].max_column)+'3')]:
            for service in services:
                listeservice.append(service.value)
                numerocol.append([service.value, service.column, name])        
            nomdoublon=[item for item, count in collections.Counter(listeservice).items() if count > 1]
            listedoublong=[nc for nc in numerocol if nc[0] in nomdoublon]
            #countdoublon=[count[1] for count in  nomdoublon
            if len(listedoublong)!=0:
                df_doublon=pd.concat([df_doublon,pd.DataFrame(listedoublong).rename(columns={0:'nomDoubon', 
                                                                                                 1:'numeroColonne',
                                                                                                 2:'annee'})])
        listeservice=[]
        numerocol=[]
    
    if len(df_doublon) ==0:
        return 'Il n\'y a pas de doublons dans les noms de service sur une année'
    else:
        return 'voici la liste des doublons', df_doublon


In [54]:
serviceDoublons()

"Il n'y a pas de doublons dans les noms de service sur une année"

In [55]:
# check libelle et codeindex identiques pour tous les onglets
def checkLibelle(fichier_excel):
    liste_probleme=[]
    df_check=pd.DataFrame({'Code index':[],
                          'libelle':[]})
    # on concat la 1ere année 2012
    # pour éviter multiindexage on ne retient que les variables service (header 2 pour police)
    df=pd.read_excel(fichier_excel,sheet_name=liste[0], header=2).rename(columns={'Libellé index \\ CGD':'libelle'})
    df=df.iloc[:,[0,1]] # df avec uniquement colonne code index et libelle
    df_check=pd.concat([df_check, df])
    
    # on concat df check avec les années suivantes
    # on sépare les 2 process de concat pour que la comparaison se fasse à partir de la 2eme année
    for sheetname in liste[1::]:
         # on garde uniquement variable service pour éviter multiindexage (header=2 pour police)
        df=pd.read_excel(fichier_excel,sheet_name=sheetname, header=2).rename(columns={'Libellé index \\ CGD':'libelle'})
        df=df.iloc[:,[0,1]] # df avec uniquement colonne code index et libelle
        df_check=pd.concat([df_check, df])
        if len(df_check[['Code index', 'libelle']].drop_duplicates(keep=False))>0:
            liste_probleme.append([str(sheetname), df_check[['Code index', 'libelle']].drop_duplicates(keep=False)])
        
    return liste_probleme

In [56]:
checkLibelle(wbpd)

[]

## <div class="alert alert-info"> 3. Structuration table 'départements' </div> 

In [85]:
def tableDep():
    #récupération fichier départements
    url = 'https://www.data.gouv.fr/fr/datasets/r/0e016380-b70b-4443-a777-16f0bec7373e'
    r = requests.get(url, allow_redirects=True)
    with open('departements.zip', 'wb') as dd:
        file=dd.write(r.content)

    # unzip file
    with ZipFile('departements.zip', 'r') as zip:
        zip.extractall('departements')
    
    # création df departement de référence avec code et nom
    df_ref_dep = pd.read_csv('departements/csv/departments.csv', 
                     usecols=['region_code','code', 'name']).rename(columns={'code':'numDep',
                                                         'name':'nomDep',
                                                         'region_code':'idRegion'})
    
    return df_ref_dep
    
    

In [86]:
df_table_dep=tableDep()

In [91]:
df_table_dep.head()

,idRegion,numDep,nomDep
0,84,01,Ain
1,32,02,Aisne
2,84,03,Allier
3,93,04,Alpes-de-Haute-Provence
4,93,05,Hautes-Alpes


In [413]:
#export csv pour sql import
try:
    df_table_dep.to_csv('fichiersPourImportSQL/export_departements.csv', index=False)
except:
    print('Problème dans  l\'export du fichier csv département.')
else:
    print('La table \'département\' bien exportée en csv')


La table 'département' bien exportée en csv


## <div class="alert alert-info"> 4. Structuration table 'region' </div> 

In [389]:
def tableRegion():
    # création df region de référence avec code et nom
    df_ref_reg = pd.read_csv('departements/csv/regions.csv', 
                     usecols=['code', 'name']).rename(columns={'code':'idRegion',
                                                               'name':'nomRegion'
    })
    
    return df_ref_reg
    
    

In [408]:
df_table_region=tableRegion()

In [409]:
df_table_region.head()

,idRegion,nomRegion
0,01,Guadeloupe
1,02,Martinique
2,03,Guyane
3,04,La Réunion
4,06,Mayotte


In [410]:
#export csv pour sql import
try:
    df_table_region.to_csv('fichiersPourImportSQL/export_regions.csv', index=False)
except:
    print('Problème dans  l\'export du fichier csv région.')
else:
    print('La table \'région\' a bien exportée en csv')

La table 'région' a bien exportée en csv


## <div class="alert alert-info"> 5. Structuration table 'direction' </div> 

In [6]:
# df de directions uniques
def dirUnique():
    liste_direction_unique = []
    for year in liste:
        for dir in police[year]['direction']:
            if dir not in liste_direction_unique:
                liste_direction_unique.append(dir)

            
    return pd.DataFrame(liste_direction_unique).rename(columns={0:'nomDirection'})

In [7]:
dirUnique()

,nomDirection
0,DCPAF
1,DCSP
2,DCPJ
3,DOPC
4,DRPJ
5,DSPAP
6,DCCRS
7,DIRECTION ORDRE PUBLIC ET CIRCULATION
8,DIRECTION REGIONALE PJ PARIS
9,DIRECTION SERVICES TECHN ET LOGISTIQUES


In [27]:
def tableDirection():
    df_dir=dirUnique()
    nomComplet_liste = ['Direction centrale de la Police aux frontières',
                        'Direction centrale de la Sécurité publique',
                        'Direction centrale de la Police judiciaire',
                        'DIRECTION ORDRE PUBLIC ET CIRCULATION',
                        'DIRECTION REGIONALE PJ PARIS',
                        'Direction de la sécurité de proximité de l\'agglomération parisienne',
                        'Direction centrale des compagnies républicaines de sécurité',
                        'DIRECTION SERVICES TECHN ET LOGISTIQUES']
    
    #renommage en sigles pour doublons (dopc et drpj existent à la fois en mode sigle et nom complet)
    df_dir.nomDirection=df_dir.nomDirection.apply(lambda x: 'DOPC' if x=='DIRECTION ORDRE PUBLIC ET CIRCULATION'
                                                      else ('DRPJ' if x=='DIRECTION REGIONALE PJ PARIS'
                                                            else'DOSTL' if x=='DIRECTION SERVICES TECHN ET LOGISTIQUES'
                                                            else x))

    #retrait doublons suite à modification de nom
    df_dir = df_dir.drop_duplicates(keep='first').dropna(axis=0).reset_index(drop=True)
    
    #colonne dirNomComplet:
    df_dir['nomComplet']=[n for n in nomComplet_liste]
    
    #colonne idAdm (1 pour police):
    df_dir['idAdm']=1
    
    return df_dir

In [28]:
df_table_direction=tableDirection()
df_table_direction

,nomDirection,nomComplet,idAdm
0,DCPAF,Direction centrale de la Police aux frontières,1
1,DCSP,Direction centrale de la Sécurité publique,1
2,DCPJ,Direction centrale de la Police judiciaire,1
3,DOPC,DIRECTION ORDRE PUBLIC ET CIRCULATION,1
4,DRPJ,DIRECTION REGIONALE PJ PARIS,1
5,DSPAP,Direction de la sécurité de proximité de l'agg...,1
6,DCCRS,Direction centrale des compagnies républicaine...,1
7,DOSTL,DIRECTION SERVICES TECHN ET LOGISTIQUES,1


In [29]:
#export csv pour sql import
try:
    df_table_direction.to_csv('fichiersPourImportSQL/export_directions.csv', index=False)
except:
    print('Problème dans  l\'export du fichier csv direction.')
else:
    print('La table \'direction\' a bien exportée en csv ')


La table 'direction' a bien exportée en csv 


## <div class="alert alert-info"> 6. Structuration table 'administration' </div> 

In [403]:
def tableAdministration():
    
    #creation df organe
    df_administration= pd.DataFrame({
        'nomAdm':['Police', 'Gendarmerie']
    })
    
    return df_administration
    
    

In [404]:
df_table_administration=tableAdministration()
df_table_administration

,nomAdm
0,Police
1,Gendarmerie


In [405]:
#export csv pour sql import
try:
    df_table_administration.to_csv('fichiersPourImportSQL/export_administration.csv', index=False)
except:
    print('Problème dans  l\'export du fichier csv administration.')
else:
    print('La table \'administration\' a bien exportée en csv: ')


La table 'administration' a bien exportée en csv: 


## <div class="alert alert-info"> 7. Structuration table 'service' </div> 

In [201]:
def serviceUnique():
    #creation liste de tous les services uniques depuis 2012
    liste_service_unique = []
    for year in liste: #liste=liste des keys dans dict police
        for serv in police[year]['service']:
            if serv not in liste_service_unique:
                liste_service_unique.append(serv)

                #df service intermédiaire
                df_service_def = pd.DataFrame(liste_service_unique).reset_index().rename(columns=
                                                                            {'index':'idService', 0:'nomService'})
                #pour être raccord avec autincrementation dans SQL qui commence à 1 et non 0 :
                df_service_def['idService'] = df_service_def['idService']+1
    return df_service_def

In [414]:
df_services_unique=serviceUnique() 
df_services_unique.head()

,idService,nomService
0,1,DIDPAF PREVESSIN 01
1,2,CIAT DE BOURG EN BRESSE
2,3,CIAT D'OYONNAX
3,4,CIAT DE CHATEAU THIERRY
4,5,CIAT DE LAON


In [203]:
def depDirServList():   
    # liste ordonnee departement, direction, serivce de toutes les années cumulées. Une sous-liste par année
    liste_triplets_total = []
    for name in police:
        indices = [i for i in range(3, wb[name].max_column+1)]
        indices_letter = [cell.get_column_letter(i) for i in indices]
        list_temp = [(wb[name][v+'1':v+'3'][i][0].value) for v in indices_letter for i in range(len(wb[name][v+'1':v+'3']))]
        liste_triplets_total.append(list_temp)
    return liste_triplets_total

In [532]:
liste_dep_dir_Serv=depDirServList()
liste_dep_dir_Serv[0][:6]

['01', 'DCPAF', 'DIDPAF PREVESSIN 01', '01', 'DCSP', 'CIAT DE BOURG EN BRESSE']

In [418]:
def CombinaisonUniqueService():
    # DF vierge
    df_triplets = pd.DataFrame({
        'numDep':[],
        'nomDirection':[],
        'nomService':[]
    })

    # creation liste des dataframe des combinaisons uniques dep-direction-service pour chaque années
    # ATTENTION : doublons présents à éliminer après pour avoir uniquement des combinaisons uniques
    frames_triplets = []
    for l in liste_dep_dir_Serv:
        subList = [l[n:n+3] for n in range(0, len(l), 3)] #sous-liste des conbinaisons dep-dir-service
        df_triple_temp = pd.DataFrame(subList).rename(columns={0:'numDep', 1:'nomDirection', 2:'nomService'})
        frames_triplets.append(df_triple_temp)

        # assemblage des dataframe de la liste
        df_triplets = pd.concat(frames_triplets) #assemblage de tous les df en un seul
        
        # suppression des duplicate rows
        df_triplets_def = df_triplets.drop_duplicates(keep='first').dropna(axis=0) 
        #keep first pour garder uniquement valeur du 1er duplicate
    #print(f'longeur du dataframe avec suppression doublon : {len(df_triplets)}\nlongeur du dataframe avec suppression doublon : {len(df_triplets_def)}')
    
    return df_triplets_def

In [423]:
df_combinaisons_uniques=CombinaisonUniqueService()
df_combinaisons_uniques.head()

,numDep,nomDirection,nomService
0,01,DCPAF,DIDPAF PREVESSIN 01
1,01,DCSP,CIAT DE BOURG EN BRESSE
2,01,DCSP,CIAT D'OYONNAX
3,02,DCSP,CIAT DE CHATEAU THIERRY
4,02,DCSP,CIAT DE LAON


In [421]:
def verifNombreService(df):
    
    nombre_service_unique = len(df_services_unique)
    nombre_doublons_df = len(df[df.duplicated('nomService')])
    nombre_total_combinaisons_unique = len(df)
    
    print(
        f'Nombre de services uniques = {nombre_service_unique}\n'
    f'Nombre de doublons de services dans DF combinaisons uniques = {nombre_doublons_df}\n'
    f'Nombre total de combinaisons uniques dep-dir-service = {nombre_total_combinaisons_unique}\n')
    
    if nombre_total_combinaisons_unique - nombre_doublons_df == nombre_service_unique:
        return 'Nous avons bien le bon nombre de services uniques dans dans ls DF combinaisons uniques'
    else:
        return 'Il y a une erreur dans le nombre de services uniques. Pas raccord avec serviceUnique()'

In [424]:
verifNombreService(df_combinaisons_uniques)

Nombre de services uniques = 814
Nombre de doublons de services dans DF combinaisons uniques = 30
Nombre total de combinaisons uniques dep-dir-service = 844



'Nous avons bien le bon nombre de services uniques dans dans ls DF combinaisons uniques'

In [425]:
def serviceMergeIdDirection():
    
    # CONVERSION noms complet de nom service en sigle pour éviter doublons avec sigles
    df_combinaisons_uniques.nomDirection=df_combinaisons_uniques.nomDirection.apply(lambda x: 'DOPC' if x=='DIRECTION ORDRE PUBLIC ET CIRCULATION'
                                                                                    else ('DRPJ' if x=='DIRECTION REGIONALE PJ PARIS'
                                                                                          else'DOSTL' if x=='DIRECTION SERVICES TECHN ET LOGISTIQUES'
                                                                                          else x))

    #reset index du DF direction pour avoir uns colonne idDirection
    df_direction=tableDirection()
    df_direction=df_direction.reset_index()

    # renommage colonne index pour idDirection
    df_direction.rename(columns={'index':'idDirection'},inplace=True)

    # ajout +1 à colonne id pour synchro avec autoincrementation SQL
    df_direction.idDirection= df_direction.idDirection+1
    
    # left join entre df triplets et direction
    df_tableService=df_combinaisons_uniques.merge(df_direction, 
                                                  how='left', 
                                                  left_on='nomDirection', 
                                                  right_on='nomDirection')
    
    return df_tableService

In [426]:
df_service_Merge_IdDirection=serviceMergeIdDirection()
df_service_Merge_IdDirection.head()

,numDep,nomDirection,nomService,idDirection,nomComplet,idAdm
0,01,DCPAF,DIDPAF PREVESSIN 01,1,Direction centrale de la Police aux frontières,1
1,01,DCSP,CIAT DE BOURG EN BRESSE,2,Direction centrale de la Sécurité publique,1
2,01,DCSP,CIAT D'OYONNAX,2,Direction centrale de la Sécurité publique,1
3,02,DCSP,CIAT DE CHATEAU THIERRY,2,Direction centrale de la Sécurité publique,1
4,02,DCSP,CIAT DE LAON,2,Direction centrale de la Sécurité publique,1


In [427]:
def tableServices():
    
    #left join entre df tableservice et nom service
    df_tableService=df_service_Merge_IdDirection.merge(df_services_unique, 
                                                  how='left', 
                                                  left_on='nomService', 
                                                  right_on='nomService')

    # suppression colonnes nomdirection car FK avec idDirection et suppression col idervice car PK AutoIncrement dans sql
    df_tableService=df_tableService.drop(['nomDirection', 'idService', 'nomComplet'], axis=1)
    
    #suppression doublons
    df_tableService = df_tableService.drop_duplicates(keep='first').dropna(axis=0)
    
    return df_tableService

In [428]:
df_table_service=tableServices()
df_table_service.head()

,numDep,nomService,idDirection,idAdm
0,01,DIDPAF PREVESSIN 01,1,1
1,01,CIAT DE BOURG EN BRESSE,2,1
2,01,CIAT D'OYONNAX,2,1
3,02,CIAT DE CHATEAU THIERRY,2,1
4,02,CIAT DE LAON,2,1


In [429]:
#verification doublons
df_table_service[df_table_service.duplicated()]

,numDep,nomService,idDirection,idAdm


In [544]:
# export csv pour import SQL
service_to_sql = df_table_service.to_csv('fichiersPourImportSQL/export_services_police.csv', index=False)


## <div class="alert alert-info"> 8. Structuration table 'pointage' </div> 

In [536]:
def tablePointage():
    # Création template DF final
    df_pointage=pd.DataFrame({
        'codeIndex':[],
        'nomService':[],
        'nbreInfractions':[],
        'annee':[]
    })

    # liste pour accueillir df restructurés par année
    df_pointage_liste = []

    # loop pour aller récupérer dans chaque onglet police le nbre d'infractions / service/ code index ss orme de DF
    for name in police:
        df_temp = pd.read_excel('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx',
                                sheet_name=name,
                                header=[2], #ne retenir que la ligne 'services'
                                # on ne garde que code index et pas libelle + services en ligne et aggregations nbre d'ifnractions:
                                index_col=0).drop('Libellé index \ CSP', axis=1).stack().reset_index().rename(
            columns={'Code index':'codeIndex', 'level_1':'nomService', 0:'nbreInfractions'})
        df_temp['annee']=str(name[-4:]) #récupération année à partir noms onglets et mise en colonne
        df_pointage_liste.append(df_temp)

    # concatener tous les df restructurés de chaque année
    df_pointage=pd.concat(df_pointage_liste)

    
    return df_pointage

In [539]:
df_table_pointage = tablePointage()


In [540]:
df_table_pointage.tail()

,codeIndex,nomService,nbreInfractions,annee
72648,107,SPAF TAHITI FAAA,1,2019
72649,107,CIAT DE PAPEETE,82,2019
72650,107,DPAF UJI 988,1,2019
72651,107,SPAF NOUMEA LA TONTOUTA,0,2019
72652,107,CIAT DE NOUMEA,321,2019


In [541]:
#verification doublons
df_table_pointage[df_table_pointage.duplicated()]

,codeIndex,nomService,nbreInfractions,annee


In [542]:
# verifications valeurs nulle:
df_table_pointage[df_table_pointage.nbreInfractions.isnull()]

,codeIndex,nomService,nbreInfractions,annee


In [543]:
# export CSV
pointage_to_sql = df_table_pointage.to_csv('fichiersPourImportSQL/export_pointage_police.csv', index=False)

In [ ]:
# merge entre nom services et idService du df_service_niques et drop colonne nomService 
    #df_pointage_def=df_pointage.merge(df_services_unique, 
                                          #how='left', 
                                          #left_on='nomService', 
                                          #right_on='nomService').drop('nomService', axis=1)


## <div class="alert alert-info"> 9. Structuration table 'indicateurs' </div> 

In [11]:
df_noms_indicateurs = pd.read_csv('indicateurs_unites/indicateurs_liste.csv')
df_noms_indicateurs

,nomIndic
0,Homicides (y compris coups et blessures volont...
1,"Vols avec armes (armes à feu, armes blanches o..."
2,Vols violents sans arme
3,Vols sans violence contre des personnes
4,Coups et blessures volontaires (sur personnes ...
5,Cambriolages de logement
6,Vols de véhicules (automobiles ou deux roues m...
7,Vols dans les véhicules
8,Vols d’accessoires sur véhicules
9,Violences sexuelles


In [12]:
def infrarename(x):
            infraRename = [
                'Cambriolages de logement',
                'Coups et blessures volontaires',
                'Destruct. et dégrad. volont.',
                'Escroqueries',
                'Homicides',
                'Violences sexuelles',
                'Vols avec armes',
                'Vols dans les véhicules',
                'Vols de véhicules',
                'Vols d’accessoires sur véh.',
                'Vols sans violence c/ des pers.',
                'Vols violents sans arme '
            ]
            infraRename.sort()
        
            a = df_noms_indicateurs.nomIndic.tolist()
            a.sort()
            f = list(zip(a, infraRename))
        
            for i in range(len(f)):
                if x == f[i][0]:
                    x = f[i][1]
                else:
                    x = x
            return x
        

In [13]:
df_noms_indicateurs['nomIndicLight'] = df_noms_indicateurs.nomIndic.apply(lambda x: infrarename(x))

In [14]:
df_noms_indicateurs

,nomIndic,nomIndicLight
0,Homicides (y compris coups et blessures volont...,Homicides
1,"Vols avec armes (armes à feu, armes blanches o...",Vols avec armes
2,Vols violents sans arme,Vols violents sans arme
3,Vols sans violence contre des personnes,Vols sans violence c/ des pers.
4,Coups et blessures volontaires (sur personnes ...,Coups et blessures volontaires
5,Cambriolages de logement,Cambriolages de logement
6,Vols de véhicules (automobiles ou deux roues m...,Vols de véhicules
7,Vols dans les véhicules,Vols dans les véhicules
8,Vols d’accessoires sur véhicules,Vols d’accessoires sur véh.
9,Violences sexuelles,Violences sexuelles


In [15]:
def tableIndicateurs():
    df_noms_indicateurs_def=df_noms_indicateurs.reset_index().rename(columns=
                                                                 {'index':'idIndic'})

    df_noms_indicateurs_def.idIndic= df_noms_indicateurs_def.idIndic.apply(lambda x: x+1)
    
    return df_noms_indicateurs_def

In [16]:
df_table_indicateur = tableIndicateurs()
df_table_indicateur

,idIndic,nomIndic,nomIndicLight
0,1,Homicides (y compris coups et blessures volont...,Homicides
1,2,"Vols avec armes (armes à feu, armes blanches o...",Vols avec armes
2,3,Vols violents sans arme,Vols violents sans arme
3,4,Vols sans violence contre des personnes,Vols sans violence c/ des pers.
4,5,Coups et blessures volontaires (sur personnes ...,Coups et blessures volontaires
5,6,Cambriolages de logement,Cambriolages de logement
6,7,Vols de véhicules (automobiles ou deux roues m...,Vols de véhicules
7,8,Vols dans les véhicules,Vols dans les véhicules
8,9,Vols d’accessoires sur véhicules,Vols d’accessoires sur véh.
9,10,Violences sexuelles,Violences sexuelles


In [18]:
# export CSV sans solonne id (généré automatiquement dans SQL)
indicateur_to_sql = df_table_indicateur.to_csv(
    'fichiersPourImportSQL/export_indicateurs.csv', columns= ['nomIndic', 'nomIndicLight'],index=False)



## <div class="alert alert-info"> 10. Structuration table 'UniteCompte' </div> 

In [509]:
#df des unités de comptes reliés à leur code index correspondant
df_uc_index=pd.read_csv('indicateurs_unites/unitesDeComptes.csv', skiprows=2, usecols=[0,2]).rename(columns={
    'Numéro de l\'index':'codeIndex', 'Unités de compte':'nomUc'
})

In [510]:
df_uc_index.head()

,codeIndex,nomUc
0,1,Victime
1,2,Victime
2,3,Victime
3,4,Victime
4,5,Victime


In [511]:
def tableUniteCompteUnique():
    # creation df des untiés de comptes uniques
    df_uc=pd.DataFrame({'nomUc':list(set(df_uc_index.nomUc))}).reset_index().rename(columns={'index':'idUc'})
    
    #ajout +1 ) colonnes index pour raccord autoincrement sql
    df_uc.idUc=df_uc.idUc+1
   
    return df_uc

In [512]:
tableUniteCompteUnique()

,idUc,nomUc
0,1,Victime entendue
1,2,Plaignant
2,3,Véhicule
3,4,Chèque
4,5,Infraction
5,6,Victime
6,7,Procédure
7,8,Auteur


In [513]:
def tableUniteCompteMergeIndex():
    
    #merge df unités de comptes uniques avec code index
    df_uc_index_def=df_uc_index.merge(tableUniteCompteUnique(), how='left', left_on='nomUc', right_on='nomUc')
    return df_uc_index_def

In [514]:
tableUniteCompteMergeIndex()

,codeIndex,nomUc,idUc
0,1,Victime,6
1,2,Victime,6
2,3,Victime,6
3,4,Victime,6
4,5,Victime,6
...,...,...,...
98,103,Procédure,7
99,104,Procédure,7
100,105,Procédure,7
101,106,Procédure,7


In [515]:
# export CSV uniquement table unite de comptes uniques
uniteComptes_to_sql = tableUniteCompteUnique().to_csv(
    'fichiersPourImportSQL/export_uniteCompte.csv', columns= ['nomUc'],index=False)




## <div class="alert alert-info"> 11. Structuration table 'infractions' </div> 

In [454]:
# connexion indicateurs Id avec code index correspondant
indicateurs={
    1:[1,2,3,6,51],
    2:[15,16,17,18,19,20,21,22],
    3:[23,24,25,26],
    4:[32,42,43],
    5:[7],
    6:[27,28],
    7:[34,35,36],
    8:[37],
    9:[38],
    10:[46,47,48,49],
    11:[84,85,86,89,90,91,92],
    12:[62,63,64,65,66,67,68]
    
}

In [471]:
def infractionsUnique():
    # les noms infraction et leur index = nomenclature unique pour chaque année. Je pars donc d'une seule anne
    # pour créer ma liste unique d'infractions
    df = pd.read_excel('crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx',
                  sheet_name='Services PN 2019',
                  usecols='A:B',
                   skiprows=[0,1]
                  )   
    df=df.rename(columns={'Code index':'codeIndex',
                         'Libellé index \ CSP':'libelle'})
    
    return df

In [472]:
df_infractions_uniques=infractionsUnique()


,codeIndex,libelle
0,1,Règlements de compte entre malfaireurs
1,2,Homicides pour voler et à l'occasion de vols
2,3,Homicides pour d'autres motifs
3,4,Tentatives d'homicides pour voler et à l'occas...
4,5,Tentatives homicides pour d'autres motifs


In [473]:
len(df_infractions_uniques)

107

In [478]:
#creation colonne IdIndic en connectant dic indicateurs et codeindex de df_infractions_uniques 
def infractionsIdIndic():
    df_infractions_uniques['idIndic']=''

    for i in range(len(df_infractions_uniques)):
        for key, value in indicateurs.items():
            if df_infractions_uniques.iloc[i,0] in value:
                df_infractions_uniques.iloc[i,2]=key
    return df_infractions_uniques

In [516]:
df_infraction_idIndic=infractionsIdIndic()


In [517]:
df_infraction_idIndic[df_infraction_idIndic.idIndic!=''].head()

,codeIndex,libelle,idIndic
0,1,Règlements de compte entre malfaireurs,1
1,2,Homicides pour voler et à l'occasion de vols,1
2,3,Homicides pour d'autres motifs,1
5,6,Coups et blessures volontaires suivis de mort,1
6,7,Autres coups et blessures volontaires criminel...,5


In [522]:
def tableInfraction():
    df_infractions_def=df_infraction_idIndic.merge(tableUniteCompteMergeIndex(), 
                                                   how='left',
                                                   left_on='codeIndex', 
                                                   right_on='codeIndex').drop('nomUc', axis=1)
    # convertir nan values en 0.0 
    df_infractions_def.idUc.fillna(value=0.0, inplace=True)
    
    # conversion colonne en int
    df_infractions_def.idUc=df_infractions_def.idUc.apply(lambda x: int(x))
    
    # remplacement 0 par ''
    df_infractions_def.idUc=df_infractions_def.idUc.apply(lambda x: '' if  x==0 else x)
    
    return df_infractions_def 

In [523]:
df_table_infraction=tableInfraction()

In [524]:
df_table_infraction.head()

,codeIndex,libelle,idIndic,idUc
0,1,Règlements de compte entre malfaireurs,1,6
1,2,Homicides pour voler et à l'occasion de vols,1,6
2,3,Homicides pour d'autres motifs,1,6
3,4,Tentatives d'homicides pour voler et à l'occas...,,6
4,5,Tentatives homicides pour d'autres motifs,,6


In [526]:
#export csv 
infractions_to_sql=df_table_infraction.to_csv('fichiersPourImportSQL/export_infractions.csv', index=False)

## <div class="alert alert-info"> 12. Structuration table 'démographie' </div> 

In [6]:
df_pop_2016 = pd.read_csv('../Demographie/data_population_densite_dep.csv', sep=';', header=[1], skiprows=1)

In [7]:
df_pop_2016.describe()

,Population au dernier recensement 2016,Densité de population 2016
count,1.000000e+02,100.000000
mean,6.636166e+05,551.240000
std,5.060584e+05,2398.170308
min,7.642200e+04,3.200000
25%,3.016595e+05,49.950000
50%,5.346745e+05,85.400000
75%,8.308875e+05,170.550000
max,2.603723e+06,20860.300000


In [8]:
df_pop_2016.tail()

,Code,Libellé,Population au dernier recensement 2016,Densité de population 2016
96,971,Guadeloupe,394110.0,242.2
97,972,Martinique,376480.0,334.1
98,973,Guyane,269352.0,3.2
99,974,Réunion,852924.0,340.6
100,976,Mayotte,NaN,NaN


In [9]:
def tableDemographie():
    #renommer colonnes en raccord avec mpd
    df_pop=df_pop_2016.rename(columns={
    'Code':'numDep',
    'Libellé':'nomDep',
    'Densité de population 2016':'densite',
    'Population au dernier recensement 2016':'nbrePopulation'
})
    #suppression colonne inutile
    df_pop=df_pop.drop('nomDep', axis=1)
    
    #remplacement na par 0 pour éviter décimal (ATTENTION: REMPLACER CETTE VALEUR PAR NULL DANS SQL)
    df_pop.nbrePopulation=df_pop.nbrePopulation.fillna(0)
    
    #remplacement na par 0 pour éviter décimal (ATTENTION: REMPLACER CETTE VALEUR PAR NULL DANS SQL)
    df_pop.densite=df_pop.densite.fillna(0)
    
    #conversion colonne nbrepopulation en entier
    df_pop.nbrePopulation=df_pop.nbrePopulation.apply(lambda x:int(x))
    
    #création colonne année de recensement
    df_pop['anneeRecensement']=2016
    
    return df_pop

In [10]:
df_table_demographie = tableDemographie()

In [11]:
df_table_demographie.tail()

,numDep,nbrePopulation,densite,anneeRecensement
96,971,394110,242.2,2016
97,972,376480,334.1,2016
98,973,269352,3.2,2016
99,974,852924,340.6,2016
100,976,0,0.0,2016


## <div class="alert alert-info"> 13. Structuration table 'démographie' - Update 1 </div> 

In [12]:
# repartion ville par dep
df_villes = pd.read_excel('https://www.insee.fr/fr/statistiques/fichier/2012729/TCRD_043.xls', 
                          header=3,
                         skipfooter=1).rename(columns={
    'Unnamed: 0':'numDep',
    'Unnamed: 1': 'nomDep',
    'Nombre de communes':'nbreCommunes',
    'Part des com. de moins de 200 hab. (en %)':'partCom_moins_200',
    'Part de la pop. vivant dans des com. de moins de 200 hab. (en %)':'partPop_moins_200',
    'Part des com. de 200 à 9999 hab. (en %)':'partCom_200_9999',
    'Part de la pop. vivant dans des com. de 200 à 9999 hab. (en %)':'partPop_200_9999',
    'Part des com. de 10000 hab. ou plus (en %)': 'partCom_10000_plus',
    'Part de la pop. vivant dans des com. de 10000 hab. ou plus (en %)':'partPop_10000_plus'
    
})

In [13]:
df_villes.columns

Index(['numDep', 'nomDep', 'nbreCommunes', 'partCom_moins_200',
       'partPop_moins_200', 'partCom_200_9999', 'partPop_200_9999',
       'partCom_10000_plus', 'partPop_10000_plus'],
      dtype='object')

In [14]:
df_villes[(df_villes.numDep=='F') | (df_villes.numDep=='M')]

,numDep,nomDep,nbreCommunes,partCom_moins_200,partPop_moins_200,partCom_200_9999,partPop_200_9999,partCom_10000_plus,partPop_10000_plus
96,M,France métropolitaine,34841,25.2,1.6,72.1,49.3,2.7,49.1
101,F,France hors Mayotte,34953,25.1,1.5,72.0,48.3,2.8,50.1


In [9]:
# drop lignes inutiles (F etM)
df_villes=df_villes.drop([101,96], axis=0)

In [21]:
#comparaison list dep avec df suivant
set(df_age_pop.numDep.unique())-set(df_villes.numDep.unique())

{'976'}

In [44]:
set(df_table_demographie.numDep.unique())-set(df_villes.numDep.unique())

{'976'}

In [15]:
# repartition population par age
df_age_pop=pd.read_csv('../Demographie/age_pop_dep_OBSTERRITOIRE.csv', sep=';', header=2).rename(columns={
    'Code':'numDep',
    'Libellé':'nomDep',
    'Part des moins de 15 ans 2016':'age_15',
    'Part des 15-29 ans 2016':'age_15_29',
    'Part des 30-44 ans 2016':'age_30_44',
    'Part des 45-59 ans 2016':'age_45_59',
    'Part des 60-74 ans 2016':'age_60_74',
    'Part des 75 ans et plus 2016':'age_75',
    'Part des moins de 20 ans 2016':'age_20'
})

In [16]:
set(df_table_demographie.numDep.unique())-set(df_age_pop.numDep.unique())

set()

In [17]:
# 1er merge entre les 2 nouvelles tables
df_merge_age_villes=df_age_pop.drop('nomDep',axis=1).merge(df_villes.drop('nomDep', axis=1), 
                                                                          how='left', 
                                                                          on='numDep')

In [18]:
df_merge_age_villes.columns

Index(['numDep', 'age_15', 'age_15_29', 'age_30_44', 'age_45_59', 'age_60_74',
       'age_75', 'age_20', 'nbreCommunes', 'partCom_moins_200',
       'partPop_moins_200', 'partCom_200_9999', 'partPop_200_9999',
       'partCom_10000_plus', 'partPop_10000_plus'],
      dtype='object')

In [19]:
# on remplace na par 0
df_merge_age_villes=df_merge_age_villes.fillna(0)

df_merge_age_villes.tail()


,numDep,age_15,age_15_29,age_30_44,age_45_59,age_60_74,age_75,age_20,nbreCommunes,partCom_moins_200,partPop_moins_200,partCom_200_9999,partPop_200_9999,partCom_10000_plus,partPop_10000_plus
96,971,19.4,16.7,17.3,23.1,15.9,7.6,26.6,32.0,0.0,0.0,56.2,22.6,43.8,77.4
97,972,17.3,16.1,16.3,24.5,16.7,9.1,23.9,34.0,0.0,0.0,67.6,28.8,32.4,71.2
98,973,33.1,23.5,21.2,14.3,6.3,1.7,42.3,22.0,4.5,0.1,59.1,16.2,36.4,83.7
99,974,23.4,20.3,19.6,21.0,11.3,4.3,31.4,24.0,0.0,0.0,29.2,5.3,70.8,94.7
100,976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# merge 2 avec table demographie n°1
df_tableDemographie_update1=df_table_demographie.merge(df_merge_age_villes, how='left', on='numDep')
df_tableDemographie_update1.head()

,numDep,nbrePopulation,densite,anneeRecensement,age_15,age_15_29,age_30_44,age_45_59,age_60_74,age_75,age_20,nbreCommunes,partCom_moins_200,partPop_moins_200,partCom_200_9999,partPop_200_9999,partCom_10000_plus,partPop_10000_plus
0,01,638425,110.7,2016,20.2,16.3,20.1,20.6,14.9,7.9,26.2,393.0,11.7,0.9,86.8,80.4,1.5,18.7
1,02,536136,72.8,2016,19.2,16.6,18.2,20.2,16.6,9.2,25.4,800.0,38.4,6.8,60.8,63.6,0.9,29.7
2,03,339384,46.3,2016,15.4,14.1,16.2,20.7,20.1,13.4,20.8,317.0,16.4,2.3,82.0,66.6,1.6,31.1
3,04,162565,23.5,2016,16.1,13.8,16.5,21.3,20.2,12.2,21.5,198.0,42.9,5.6,56.1,70.8,1.0,23.7
4,05,141107,25.4,2016,16.8,14.2,17.8,21.3,18.6,11.2,22.3,162.0,31.5,3.9,67.3,58.9,1.2,37.2


In [21]:
df_tableDemographie_update1.columns

Index(['numDep', 'nbrePopulation', 'densite', 'anneeRecensement', 'age_15',
       'age_15_29', 'age_30_44', 'age_45_59', 'age_60_74', 'age_75', 'age_20',
       'nbreCommunes', 'partCom_moins_200', 'partPop_moins_200',
       'partCom_200_9999', 'partPop_200_9999', 'partCom_10000_plus',
       'partPop_10000_plus'],
      dtype='object')

In [25]:
# check na values
df_tableDemographie_update1.isna().sum()

numDep                0
nbrePopulation        0
densite               0
anneeRecensement      0
age_15                0
age_15_29             0
age_30_44             0
age_45_59             0
age_60_74             0
age_75                0
age_20                0
nbreCommunes          0
partCom_moins_200     0
partPop_moins_200     0
partCom_200_9999      0
partPop_200_9999      0
partCom_10000_plus    0
partPop_10000_plus    0
dtype: int64

In [53]:
# export CSV
df_tableDemographie_update1.to_csv('fichiersPourImportSQL/export_demographieUpdate1.csv', index=False)

In [77]:
# taux de chomage par ded et par -age
df_chomage = pd.read_csv('../Demographie/chomage_dep_OBSTERRITOIRE.csv', sep=';',header=2)

In [84]:
df_chomage.tail()

,Code,Libellé,Taux de chômage des 15 ans et plus (RP) 2016,Taux de chômage des 55-64 ans (RP) 2016,Taux de chômage des 15-24 ans (RP) 2016,Part des moins de 25 ans parmi les demandeurs d'emploi de catégorie ABC 2017
96,971,Guadeloupe,29.1,19.4,52.0,11.0
97,972,Martinique,24.8,16.3,51.3,10.4
98,973,Guyane,35.2,21.7,57.6,14.1
99,974,Réunion,34.6,26.8,55.9,12.6
100,976,Mayotte,NaN,NaN,NaN,NaN
